In [1]:
%matplotlib widget

import pickle
import os
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import numpy as np
import pandas as pd

# Check Losses

In [2]:
with open(os.path.join('.', 'losses.p'), 'rb') as f:
    losses = pickle.load(f)

In [3]:
fig, ax = plt.subplots()
plt.plot(losses)
ax.set_xlabel('EM Step')
ax.set_ylabel('NLL')
ax.set_title('Synthetic Data NLL 3 dimensions')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'NLL')

# Check Dimension Correlations

In [4]:
with open(os.path.join('.', 'thetas.p'), 'rb') as f:
    thetas = pickle.load(f).detach()
    
with open(os.path.join('.', 'item_params.p'), 'rb') as f:
    item_params = pickle.load(f).detach()
    
with open(os.path.join('.', 'params_order.p'), 'rb') as f:
    param_order = pickle.load(f)

In [5]:
print(thetas.shape)
print(param_order)
print(item_params.shape)

torch.Size([20, 3])
['a', 'b']
torch.Size([3000, 2, 3])


In [6]:
a = item_params[:, 0, :]
b = item_params[:, 1, :]

In [7]:
corr = np.corrcoef(thetas.T)
print(corr)
fig, ax = plt.subplots()
sns.heatmap(corr, vmin=-1, vmax=1)
ax.set_title('Thetas')

[[ 1.         -0.27897075 -0.08539377]
 [-0.27897075  1.         -0.1526922 ]
 [-0.08539377 -0.1526922   1.        ]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1, 'Thetas')

In [8]:
corr = np.corrcoef(a.T)
print(corr)
fig, ax = plt.subplots()
sns.heatmap(corr, vmin=-1, vmax=1)
ax.set_title('Discrimination')

[[ 1.          0.02865455 -0.00453232]
 [ 0.02865455  1.         -0.01499718]
 [-0.00453232 -0.01499718  1.        ]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1, 'Discrimination')

In [9]:
corr = np.corrcoef(b.T)
print(corr)
fig, ax = plt.subplots()
sns.heatmap(corr, vmin=-1, vmax=1)
ax.set_title('Difficulty')

[[ 1.          0.04156048 -0.00528217]
 [ 0.04156048  1.          0.02755141]
 [-0.00528217  0.02755141  1.        ]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1, 'Difficulty')

# Check Reconstruction MSE

In [10]:
sync_params = os.path.join('..', 'diff_skills', 'params_sync_dim3_diffskills_irt_all_coded.p')

In [11]:
with open(sync_params, 'rb') as f:
    sync_p = pickle.load(f)

In [12]:
def mse(real, recon):
    diff = real - recon
    return (diff*diff).sum()/np.prod(real.shape)

def sigmoid(x):
    return 1/(1+np.exp(x))

def probs(theta, a, b):
    return sigmoid(
        torch.matmul(
            a[:, None, :],
            (b[:, None, :] - theta[None, :, :]).transpose(2, 1) # nitems x ndims x nresp
        )
    ).squeeze()

In [13]:
gold_resp_fname = os.path.join('..', 'diff_skills', 'sync_dim3_diffskills_irt_all_coded.csv')
gold_r = pd.read_csv(gold_resp_fname).iloc[:,1:].to_numpy()

In [14]:
pred_r = np.greater_equal(probs(thetas, a, b).numpy().T,0.5).astype('int')

In [15]:
print(f'Theta MSE {mse(sync_p["t"], thetas.numpy())}')
print(f'Alpha MSE {mse(sync_p["a"], a.numpy())}')
print(f'Beta MSE {mse(sync_p["b"], b.numpy())}')
print(f'Probabilities MSE {mse(sync_p["probs"], probs(thetas, a, b).numpy().T)}')

Theta MSE 26.852803548177082
Alpha MSE 1.8735948350694445
Beta MSE 78.8729236111111
Probabilities MSE 0.03329179077148438


In [16]:
print(f'Response MSE {mse(gold_r, pred_r)}')
print(gold_r.mean())

Response MSE 0.6047833333333333
0.5393666666666667


In [17]:
print(f'All True Response Baseline MSE {mse(gold_r, np.ones(gold_r.shape))}')
print(f'All False Response Baseline MSE {mse(gold_r, np.zeros(gold_r.shape))}')
print(f'Probabilities 0.5 Baseline MSE {mse(sync_p["probs"], np.ones(sync_p["probs"].shape)*0.5)}')

All True Response Baseline MSE 0.46063333333333334
All False Response Baseline MSE 0.5393666666666667
Probabilities 0.5 Baseline MSE 0.00646812606403941


In [18]:
f, a = plt.subplots()
plt.hist(sync_p["probs"].reshape((-1,1)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([47330.,  5743.,  2595.,  1499.,   985.,   652.,   487.,   345.,
          243.,   121.]),
 array([0.5000033 , 0.54951733, 0.5990313 , 0.6485454 , 0.69805944,
        0.7475735 , 0.7970875 , 0.84660155, 0.8961156 , 0.9456296 ,
        0.99514365], dtype=float32),
 <a list of 10 Patch objects>)